In [1]:
import pandas as pd
import numpy as np

# Corregir la ruta del archivo
vp_week = pd.read_excel("/home/donsson/proyectos/MODELO ABASTECIMIENTO/exceles/vp_año.xlsx") #ventas perdidas
df = pd.read_parquet("/home/donsson/proyectos/API/ventashistoricas52semanas.parquet") #movimiento  facturas
df_p = pd.read_parquet("/home/donsson/proyectos/API/costo_productos.parquet") #Costos unitarios

In [2]:
df.head()

,origin,price_subtotal,quantity,product_name,invoice_name,date_invoice,store_name
0,False,166500.0,1.0,[DAE02502025] DA2502 FILTRO AIRE PLANTAS CUMMI...,PV4E16294 Mostrador Norte T2/13113,2025-09-29 15:31:19,SUCURSAL NORTE
1,False,71200.0,1.0,[BCS30168125] GS168A1 FILTRO COMBUSTIBLE SEPAR...,PV2E43726 Mostrador Calle 6 T1/25533,2025-09-29 15:24:40,SUCURSAL CALLE 6
2,False,22400.0,1.0,[DCS00235189] GS235 FILTRO COMBUSTIBLE - TOYOT...,PV2E43726 Mostrador Calle 6 T1/25533,2025-09-29 15:24:40,SUCURSAL CALLE 6
3,False,27900.0,1.0,[DAE00706189] GA706 FILTRO AIRE - NISSAN (189...,PV4E16293 Mostrador Norte T2/13112,2025-09-29 15:13:03,SUCURSAL NORTE
4,False,906800.0,2.0,[ALB15W40131] ACEITE MOBIL MX15W40 DELVAC x GA...,PV6E19441 Mostrador Barranquilla T3/3363,2025-09-29 15:07:14,SUCURSAL BARRANQUILLA


# EDA

## Facturas

In [3]:
import re
import unicodedata



mapa_sucursales = {
    'BD1': 'PRINCIPAL COTA',
    'BD2': 'SUCURSAL CALLE 6',
    'BD3': 'SUCURSAL VALLADOLID',
    'BD4': 'SUCURSAL NORTE',
    'BD5': 'SUCURSAL MEDELLIN',
    'BD6': 'SUCURSAL BARRANQUILLA',
    'BD7':' SUCURSAL BUCARAGAMNGA ',
    'BD8':'MOSTRADOR COTA',
    'BD9':'SUCURSAL CALI',
    'BD11': 'CUMMINS DE LOS ANDES',
    'BD12': 'CUMMINS DE LOS ANDES MEDELLIN',
    'BD13': 'CSS CONSTRUCTORES',
    'BD14': 'ANTIOQUEÑA DE LUBRICANTES SGP SAS',
    'BD15': 'ASEO CAPITAL',
    'BD16': 'COVIANDES',
    'BD18': 'CEMEX BOSA',
    'BD19': 'Cemex ibague (eliminar)',
    'BD21': 'CEMEX CENTENARIO',
    'BD22': 'PENDIENTE POR FACTURAR CIERRE CONT.',
    'BD24': 'FILTRO EN REPROCESO',
    'BD26': 'PRESTAMOS INTERNOS',
    'BD31': 'EXP.CTP',
    'BD32': 'EXP. ECUADOR',
    'BD34':'EXP.DONSSON.USA',
    'BD40': 'DESCUADRE B1',
    'BD MP':'MATERIA PRIMA',
    'BD41':'BODEGA VENDEDOR WILMER GIL',
    'BD33':'EXP.REP.DOMINICANA',
    'BD42':'STOCK SIGMA ENERGY'
    # Agrega más códigos si los encuentras en tus datos, siguiendo este formato.
    }




def clasificar_sucursal(invoice_name):
    if pd.isna(invoice_name):
        return "VENDEDOR EXTERNO"
    if "Mostrador" in invoice_name:
        return "MOSTRADOR"
    return "VENDEDOR EXTERNO"
    
df["Sucursal_tipo"] = df["invoice_name"].apply(clasificar_sucursal)




#Quitar las facturas que salen como SO son no vendibles
df = df[~(df["origin"].str.contains("SO"))] 


# Normalizar los separadores para que todos sean iguales
df["origin_norm"] = df["origin"].str.replace(r"[\\/]", "/", regex=True)

# Extraer solo el BD y el número (ejemplo: BD11, BD9, etc.)
df["origen"] = df["origin_norm"].str.extract(r"^(BD\d+)")

df["Bodega"] = df["origen"].map(mapa_sucursales)
df["Bodega"] = df["Bodega"].fillna(df["store_name"])

df = df.drop(columns=["origin_norm"])


df = df.drop(columns=["origin"])

df_externos_bogota = df[(df["store_name"]=="PRINCIPAL COTA") & (df["Sucursal_tipo"]=="VENDEDOR EXTERNO")]

df = df.drop(df_externos_bogota.index)


In [4]:
df.groupby(["store_name","Bodega","Sucursal_tipo"])["quantity"].sum()

store_name             Bodega                             Sucursal_tipo   
PRINCIPAL COTA         PRINCIPAL COTA                     MOSTRADOR           15870.0
SUCURSAL BARRANQUILLA  BODEGA VENDEDOR WILMER GIL         VENDEDOR EXTERNO      113.0
                       SUCURSAL BARRANQUILLA              MOSTRADOR           24118.0
                                                          VENDEDOR EXTERNO    74345.0
                       SUCURSAL CALI                      VENDEDOR EXTERNO        2.0
SUCURSAL BUCARAMANGA    SUCURSAL BUCARAGAMNGA             VENDEDOR EXTERNO    46164.0
                       PRINCIPAL COTA                     VENDEDOR EXTERNO        2.0
                       SUCURSAL BUCARAMANGA               MOSTRADOR           18344.0
SUCURSAL CALI          BODEGA VENDEDOR WILMER GIL         VENDEDOR EXTERNO     2016.0
                       EXP.REP.DOMINICANA                 VENDEDOR EXTERNO     1961.0
                       PRINCIPAL COTA                     VENDEDO

In [5]:
df[df["product_name"].str.contains("DCE00833189")].sort_values(by=["quantity"],ascending=False)

,price_subtotal,quantity,product_name,invoice_name,date_invoice,store_name,Sucursal_tipo,origen,Bodega
10925,715000.0,50.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV2E43208 Mostrador Calle 6 T2/21164,2025-09-10 17:49:58,SUCURSAL CALLE 6,MOSTRADOR,<NA>,SUCURSAL CALLE 6
100512,715000.0,50.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,FBAQ13002,2025-04-02 21:04:23,SUCURSAL BARRANQUILLA,VENDEDOR EXTERNO,BD6,SUCURSAL BARRANQUILLA
191502,514800.0,36.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV1E8545 Mostrador Cota T2/1936,2024-10-15 20:33:42,PRINCIPAL COTA,MOSTRADOR,<NA>,PRINCIPAL COTA
57002,477000.0,30.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV2E40849 Mostrador Calle 6 T3/22423,2025-06-20 14:02:51,SUCURSAL CALLE 6,MOSTRADOR,<NA>,SUCURSAL CALLE 6
119280,540000.0,30.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV2E37805 Mostrador Calle 6 T3/21299,2025-02-28 14:32:09,SUCURSAL CALLE 6,MOSTRADOR,<NA>,SUCURSAL CALLE 6
...,...,...,...,...,...,...,...,...,...
196126,18000.0,1.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV2E33785 Mostrador Calle 6 T3/20417,2024-10-05 15:18:36,SUCURSAL CALLE 6,MOSTRADOR,<NA>,SUCURSAL CALLE 6
195869,18000.0,1.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,FNOR1439,2024-10-07 14:25:49,SUCURSAL NORTE,VENDEDOR EXTERNO,BD4,SUCURSAL NORTE
195788,18000.0,1.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,FNOR1441,2024-10-07 14:40:49,SUCURSAL NORTE,VENDEDOR EXTERNO,BD4,SUCURSAL NORTE
194528,18000.0,1.0,[DCE00833189] G833 FILTRO COMBUSTIBLE CHEVROLE...,PV2E33874 Mostrador Calle 6 T4/11693,2024-10-08 20:22:27,SUCURSAL CALLE 6,MOSTRADOR,<NA>,SUCURSAL CALLE 6


In [6]:
df_externos_bogota = df[(df["store_name"]=="SUCURSAL CALI") & (df["Bodega"]=="EXP.REP.DOMINICANA")]

df_externos_bogota

,price_subtotal,quantity,product_name,invoice_name,date_invoice,store_name,Sucursal_tipo,origen,Bodega
59841,360000.0,24.0,"[DAB04671025] DA4671 FILTRO AIRE 2_ KUBOTA, NI...",FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
59842,228000.0,4.0,[DAB08019025] DA8019 FILTRO AIRE JAC 1061 (025...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
59843,321200.0,4.0,[DAE08022025] DA8022 FILTRO AIRE - FOTON LOVOL...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
59844,168700.0,1.0,[DAR08076025] DA8076 FILTRO AIRE 1 HYUNDAI HD2...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
59845,62000.0,1.0,[DAE04416025] DA4416 FILTRO AIRE 2_ CATERPILLA...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
...,...,...,...,...,...,...,...,...,...
59913,7468750.0,125.0,[DAB02690025] DA2690 FILTRO AIRE- FREIGTHLINER...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
59914,3393750.0,125.0,[DAB02788025] DA2788 FILTRO AIRE DONSSON - Ch...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
59915,2062500.0,125.0,"[DAB02772025] DA2772 FILTRO AIRE BOBCAT, HITAC...",FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA
59916,2237500.0,125.0,[DAB02738025] DA2738 FILTRO AIRE CORTACESPED J...,FCAL4648,2025-06-16 14:09:12,SUCURSAL CALI,VENDEDOR EXTERNO,BD33,EXP.REP.DOMINICANA


# UNION

## EMA SEMANAL CON VP SEMANALES (SOLO 2025)

In [7]:
# ===============================
# Procesar ventas normales
# ===============================
df_sales = df.copy()
df_sales["date_invoice"] = pd.to_datetime(df_sales["date_invoice"], errors="coerce")


# Referncia de producto
df_sales["product_ref"] = df_sales["product_name"].str.extract(r"\[([A-Z0-9]+)\]")


# Columnas temporales igual que en df_vp
df_sales["Semana"] = df_sales["date_invoice"].dt.to_period("W").dt.start_time
df_sales["ano"]    = df_sales["Semana"].dt.year
df_sales["mes"]    = df_sales["Semana"].dt.month
df_sales["dia"]    = df_sales["Semana"].dt.day

# ===============================
# Agrupación por tienda + producto + semana
# ===============================
sales_by_week = (
    df_sales.groupby(["store_name", "product_ref", "Semana", "ano", "mes", "dia"], as_index=False)
    .agg(
        sales=("quantity", "sum"),              # suma de unidades vendidas
        num_facturas=("date_invoice", "nunique")  # cantidad de facturas distintas
    )
)


# Resultado
sales_by_week.sample(10)



,store_name,product_ref,Semana,ano,mes,dia,sales,num_facturas
49804,SUCURSAL CALLE 6,BCS00272125,2025-02-24,2025,2,24,1.0,1
61719,SUCURSAL CALLE 6,DAR03114025,2025-01-13,2025,1,13,2.0,1
51536,SUCURSAL CALLE 6,BCS10417125,2025-06-30,2025,6,30,1.0,1
41158,SUCURSAL CALI,BLS00286125,2025-03-17,2025,3,17,1.0,1
41026,SUCURSAL CALI,BLS00234125,2025-01-27,2025,1,27,2.0,1
77597,SUCURSAL MEDELLIN,DAR03750025,2025-06-09,2025,6,9,3.0,1
86031,SUCURSAL NORTE,DAE04066025,2025-03-17,2025,3,17,1.0,1
61991,SUCURSAL CALLE 6,DAR12523025,2024-12-02,2024,12,2,1.0,1
70246,SUCURSAL MEDELLIN,BLS00237125,2024-11-04,2024,11,4,1.0,1
16570,SUCURSAL BARRANQUILLA,DAB04547025,2024-11-11,2024,11,11,11.0,4


In [8]:
df_merged = pd.merge(
    sales_by_week[["store_name", "product_ref", "Semana", "sales","num_facturas"]],
    vp_week[["store_name", "product_ref", "Semana", "lost_sales","veces_vp"]],
    on=["store_name", "product_ref", "Semana"],
    how="outer"
).fillna(0)


In [9]:
df_merged["año"]    = df_merged["Semana"].dt.year
df_merged["mes"]    = df_merged["Semana"].dt.month
df_merged["dia"]    = df_merged["Semana"].dt.day


df_merged.head(10)

,store_name,product_ref,Semana,sales,num_facturas,lost_sales,veces_vp,año,mes,dia
0,PRINCIPAL COTA,AC10388020,2025-09-22,0.0,0.0,36.0,4.0,2025,9,22
1,PRINCIPAL COTA,AGB0GRAS030,2025-07-28,0.0,0.0,1.0,1.0,2025,7,28
2,PRINCIPAL COTA,AHB0TO30132,2025-01-13,0.0,0.0,1.0,1.0,2025,1,13
3,PRINCIPAL COTA,AHB80W90030,2025-07-07,0.0,0.0,1.0,1.0,2025,7,7
4,PRINCIPAL COTA,AHB80W90050,2025-08-11,0.0,0.0,1.0,1.0,2025,8,11
5,PRINCIPAL COTA,AHBNTO68030,2024-10-21,1.0,1.0,0.0,0.0,2024,10,21
6,PRINCIPAL COTA,AHBNTO68030,2025-01-27,1.0,1.0,1.0,1.0,2025,1,27
7,PRINCIPAL COTA,AHBNTO68030,2025-02-03,0.0,0.0,1.0,1.0,2025,2,3
8,PRINCIPAL COTA,AHBNTO68030,2025-02-10,0.0,0.0,1.0,1.0,2025,2,10
9,PRINCIPAL COTA,AHBNTO68030,2025-02-17,0.0,0.0,4.0,1.0,2025,2,17


In [10]:
df_merged.query("product_ref == 'BAE05001125'")

,store_name,product_ref,Semana,sales,num_facturas,lost_sales,veces_vp,año,mes,dia
15357,SUCURSAL BARRANQUILLA,BAE05001125,2025-01-20,2.0,1.0,0.0,0.0,2025,1,20
15358,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-24,0.0,0.0,1.0,1.0,2025,2,24
15359,SUCURSAL BARRANQUILLA,BAE05001125,2025-03-24,0.0,0.0,3.0,3.0,2025,3,24
15360,SUCURSAL BARRANQUILLA,BAE05001125,2025-09-08,1.0,1.0,0.0,0.0,2025,9,8
92620,SUCURSAL MEDELLIN,BAE05001125,2025-02-17,0.0,0.0,1.0,1.0,2025,2,17
92621,SUCURSAL MEDELLIN,BAE05001125,2025-02-24,0.0,0.0,1.0,1.0,2025,2,24
92622,SUCURSAL MEDELLIN,BAE05001125,2025-06-09,0.0,0.0,1.0,1.0,2025,6,9
92623,SUCURSAL MEDELLIN,BAE05001125,2025-08-18,0.0,0.0,1.0,1.0,2025,8,18
112218,SUCURSAL NORTE,BAE05001125,2025-02-24,0.0,0.0,1.0,1.0,2025,2,24


In [11]:
df_merged["product_ref"].nunique()

4310

## NORMALIZAR DF DE COSTOS

In [12]:
df_p["product_ref"] = df_p["product_name"].str.extract(r"\[([A-Z0-9]+)\]")

df_p_unique = (
    df_p[["product_ref", "producto_costo_unitario"]]
    .drop_duplicates(subset=["product_ref"])
)



df_p["product_ref"].nunique()

4835

## EMA 

In [13]:
import pandas as pd
import numpy as np

def compute_demand_and_ema(df,
                           alpha=0.20,        # peso de EMA
                           n_init_weeks=12,   # semanas para inicializar EMA
                           week_col="Semana",
                           sales_col="sales",
                           lost_col="lost_sales"):
    df = df.copy()

    # ---------- Asegurar tipos y semana iniciando lunes ----------
    df[week_col] = pd.to_datetime(df[week_col], errors="coerce")
    df[week_col] = df[week_col].dt.to_period('W-MON').dt.start_time

    # Asegurar numéricos
    df[sales_col] = pd.to_numeric(df[sales_col], errors="coerce").fillna(0)
    df[lost_col]  = pd.to_numeric(df[lost_col], errors="coerce").fillna(0)

    # ---------- Crear combinaciones completas ----------
    stores   = df["store_name"].unique()
    products = df["product_ref"].unique()


    # Crear rango completo desde min hasta max semana
    weeks = pd.period_range(df[week_col].min(), df[week_col].max(), freq="W-MON").to_timestamp()


    full_index = pd.MultiIndex.from_product([stores, products, weeks],
                                            names=["store_name", "product_ref", week_col])

    df = df.set_index(["store_name", "product_ref", week_col]).reindex(full_index).reset_index()

    # Rellenar ventas y perdidas con 0 en combinaciones faltantes
    df[sales_col] = df[sales_col].fillna(0)
    df[lost_col]  = df[lost_col].fillna(0)

    # Ordenar
    df = df.sort_values(["store_name", "product_ref", week_col])

    out_groups = []

    # ---------- Calcular demanda ajustada y EMA ----------
    for (store, prod), g in df.groupby(["store_name", "product_ref"], sort=False):
        g = g.sort_values(week_col).reset_index(drop=True)
        sales = g[sales_col].to_numpy(dtype=float)
        lost  = g[lost_col].to_numpy(dtype=float)

        L = len(g)
        demanda = np.zeros(L, dtype=float)
        ema_arr = np.zeros(L, dtype=float)

        if L == 0:
            out_groups.append(g)
            continue

        # Inicialización EMA: promedio de primeras n_init_weeks ventas
        init_n = min(n_init_weeks, L)
        ema_prev = float(np.nanmean(sales[:init_n])) if init_n > 0 else 0.0
        if np.isnan(ema_prev):
            ema_prev = 0.0

        for i in range(L):
            s = sales[i]
            l = lost[i]

            # Regla 1
            if s >= 2.0 * l:
                demand_candidate = s + l
                demand = min(demand_candidate, 1.5 * s) if s > 0 else demand_candidate
            else:
                # Regla 2
                demand = s + 0.5 * ema_prev

            # Guardar redondeando
            demanda[i] = round(demand, 6)

            # Calcular EMA
            ema = alpha * demand + (1.0 - alpha) * ema_prev
            ema = round(ema, 6)
            ema_arr[i] = ema

            # actualizar para siguiente
            ema_prev = ema

        # Añadir columnas al grupo
        g = g.copy()
        g["demanda_ajustada"] = demanda
        g["EMA"] = ema_arr

        out_groups.append(g)

        

    # Concat resultados
    result = pd.concat(out_groups, ignore_index=True, sort=False)
    result = result.sort_values(["store_name", "product_ref", week_col]).reset_index(drop=True)
    return result

# ------------------ USO ------------------
df_with_demand1 = compute_demand_and_ema(df_merged, alpha=0.2, n_init_weeks=12) #estaba en 16

#APROX3min

In [14]:
df_with_demand1.groupby("store_name")["product_ref"].nunique()

store_name
PRINCIPAL COTA           4310
SUCURSAL BARRANQUILLA    4310
SUCURSAL BUCARAMANGA     4310
SUCURSAL CALI            4310
SUCURSAL CALLE 6         4310
SUCURSAL MEDELLIN        4310
SUCURSAL NORTE           4310
SUCURSAL VALLADOLID      4310
Name: product_ref, dtype: int64

In [15]:
def add_deviation_and_classification(df, ema_col="EMA", sales_col="sales", week_col="Semana", n_weeks=12): ##PERILLA SEMANAS 
    df = df.copy()
    out_groups = []

    for (store, prod), g in df.groupby(["store_name", "product_ref"], sort=False):
        g = g.sort_values(week_col).reset_index(drop=True)
        
        sales = g[sales_col].to_numpy(dtype=float)
        ema   = g[ema_col].to_numpy(dtype=float)
        L = len(sales)

        # Inicializamos columnas
        g["desviacion_ema%"] = 0.0
        g["coef_ventas"] = 0.0

        if L < 2:
            out_groups.append(g)
            continue

        for i in range(L):
            # Usar solo hasta la semana actual (i)
            window_start = max(0, i - n_weeks)       # ventana de 12 semanas
            window_sales = sales[window_start:i]     # semanas previas
            window_ema   = ema[window_start:i]

            if len(window_sales) < 2:
                continue

            promedio_prev = np.mean(window_sales)
            if promedio_prev == 0:
                promedio_prev = 1e-6

            # --- Desviación usando EMAs previos ---
            desv = np.std(window_ema, ddof=1) / promedio_prev if promedio_prev > 0 else 0

            # --- Coeficiente de ventas ---
            coef = (max(window_sales) - min(window_sales)) / promedio_prev if promedio_prev > 0 else 0

            g.at[i, "desviacion_ema%"] = round(desv,2)
            g.at[i, "coef_ventas"] = coef

        out_groups.append(g)

    return pd.concat(out_groups, ignore_index=True)





df_with_demand2 = add_deviation_and_classification(df_with_demand1)
df_with_demand2 = df_with_demand2[["store_name","product_ref","Semana","num_facturas","sales","lost_sales","demanda_ajustada","EMA","desviacion_ema%","coef_ventas"]]

# Demora mas o menos 8 min

In [16]:
df_with_demand2.query("product_ref == 'BAE05001125'").sort_values(by=["EMA"]).tail(10)

,store_name,product_ref,Semana,num_facturas,sales,lost_sales,demanda_ajustada,EMA,desviacion_ema%,coef_ventas
252779,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-25,NaN,0.0,0.0,0.00000,0.117965,0.87,12.0
252808,SUCURSAL BARRANQUILLA,BAE05001125,2025-09-16,NaN,0.0,0.0,0.00000,0.128206,0.84,12.0
252778,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-18,0.0,0.0,1.0,0.08192,0.147456,0.90,12.0
252807,SUCURSAL BARRANQUILLA,BAE05001125,2025-09-09,NaN,0.0,0.0,0.00000,0.160257,0.69,12.0
252777,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-11,NaN,0.0,0.0,0.00000,0.163840,0.91,12.0
252806,SUCURSAL BARRANQUILLA,BAE05001125,2025-09-02,1.0,1.0,0.0,1.00000,0.200321,1378.24,0.0
252776,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-04,NaN,0.0,0.0,0.00000,0.204800,0.90,12.0
252775,SUCURSAL BARRANQUILLA,BAE05001125,2025-01-28,NaN,0.0,0.0,0.00000,0.256000,0.85,12.0
252774,SUCURSAL BARRANQUILLA,BAE05001125,2025-01-21,NaN,0.0,0.0,0.00000,0.320000,0.69,12.0
252773,SUCURSAL BARRANQUILLA,BAE05001125,2025-01-14,1.0,2.0,0.0,2.00000,0.400000,0.00,0.0


In [17]:
df_with_demand2["semana_num"] = df_with_demand2["Semana"].dt.isocalendar().week
df_with_demand2["año"] = df_with_demand2["Semana"].dt.year


### AGREGAR SEMANA CSV

In [18]:
df_demand_2025 = df_with_demand2[df_with_demand2["año"]==2025] #Ajustar año

demand_2025 = df_demand_2025[
    ["store_name","product_ref","año","Semana","semana_num",
     "num_facturas","EMA","demanda_ajustada","desviacion_ema%","coef_ventas","lost_sales"]
]


demand_2025_38 = demand_2025[demand_2025["semana_num"]==38]  #Ajustar semana numero

df_with_demand2.query("product_ref == 'BAE05001125'").sort_values(by=["EMA"]).tail(10)




,store_name,product_ref,Semana,num_facturas,sales,lost_sales,demanda_ajustada,EMA,desviacion_ema%,coef_ventas,semana_num,año
252779,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-25,NaN,0.0,0.0,0.00000,0.117965,0.87,12.0,9,2025
252808,SUCURSAL BARRANQUILLA,BAE05001125,2025-09-16,NaN,0.0,0.0,0.00000,0.128206,0.84,12.0,38,2025
252778,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-18,0.0,0.0,1.0,0.08192,0.147456,0.90,12.0,8,2025
252807,SUCURSAL BARRANQUILLA,BAE05001125,2025-09-09,NaN,0.0,0.0,0.00000,0.160257,0.69,12.0,37,2025
252777,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-11,NaN,0.0,0.0,0.00000,0.163840,0.91,12.0,7,2025
252806,SUCURSAL BARRANQUILLA,BAE05001125,2025-09-02,1.0,1.0,0.0,1.00000,0.200321,1378.24,0.0,36,2025
252776,SUCURSAL BARRANQUILLA,BAE05001125,2025-02-04,NaN,0.0,0.0,0.00000,0.204800,0.90,12.0,6,2025
252775,SUCURSAL BARRANQUILLA,BAE05001125,2025-01-28,NaN,0.0,0.0,0.00000,0.256000,0.85,12.0,5,2025
252774,SUCURSAL BARRANQUILLA,BAE05001125,2025-01-21,NaN,0.0,0.0,0.00000,0.320000,0.69,12.0,4,2025
252773,SUCURSAL BARRANQUILLA,BAE05001125,2025-01-14,1.0,2.0,0.0,2.00000,0.400000,0.00,0.0,3,2025


## UNIR COSTO

In [19]:
df_merge_def = pd.merge(
    demand_2025_38,
    df_p_unique,
    on="product_ref",
    how="outer" #para incluir todos , estaba antes left
).fillna(0)

df_merge_def.to_csv("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/ema_mio202538.csv")

#merge_def = df_merge_def[df_merge_def["producto_costo_unitario"] !=0].copy() #Eliminar productos sin costos unitarios

merge_def =  df_merge_def.copy()
merge_def.groupby("store_name")["product_ref"].nunique()

store_name
0                         565
PRINCIPAL COTA           4310
SUCURSAL BARRANQUILLA    4310
SUCURSAL BUCARAMANGA     4310
SUCURSAL CALI            4310
SUCURSAL CALLE 6         4310
SUCURSAL MEDELLIN        4310
SUCURSAL NORTE           4310
SUCURSAL VALLADOLID      4310
Name: product_ref, dtype: int64

In [20]:
merge_def[merge_def["store_name"]==0].sample(10)

,store_name,product_ref,año,Semana,semana_num,num_facturas,EMA,demanda_ajustada,desviacion_ema%,coef_ventas,lost_sales,producto_costo_unitario
34370,0,DLS00246004,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2797.50
22516,0,DAE04444024,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,22400.00
29117,0,DCE00543189,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,7594.95
23157,0,DAE05452025,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,5491.37
29020,0,DCE00525136,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,7500.00
34155,0,DLS00077010,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,18983.85
22533,0,DAE04458025,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,6016.82
29355,0,DCE00589015,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,5000.00
19625,0,DAC01072122,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,6860.74
28394,0,DCE00158015,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,1401.22


In [21]:
merge_def.query("product_ref == 'BAE05001125'")

,store_name,product_ref,año,Semana,semana_num,num_facturas,EMA,demanda_ajustada,desviacion_ema%,coef_ventas,lost_sales,producto_costo_unitario
3730,PRINCIPAL COTA,BAE05001125,2025.0,2025-09-16 00:00:00,38,0.0,0.000000,0.0,0.00,0.0,0.0,90836.27
3731,SUCURSAL BARRANQUILLA,BAE05001125,2025.0,2025-09-16 00:00:00,38,0.0,0.128206,0.0,0.84,12.0,0.0,90836.27
3732,SUCURSAL BUCARAMANGA,BAE05001125,2025.0,2025-09-16 00:00:00,38,0.0,0.000000,0.0,0.00,0.0,0.0,90836.27
3733,SUCURSAL CALI,BAE05001125,2025.0,2025-09-16 00:00:00,38,0.0,0.000000,0.0,0.00,0.0,0.0,90836.27
3734,SUCURSAL CALLE 6,BAE05001125,2025.0,2025-09-16 00:00:00,38,0.0,0.000000,0.0,0.00,0.0,0.0,90836.27
3735,SUCURSAL MEDELLIN,BAE05001125,2025.0,2025-09-16 00:00:00,38,0.0,0.000000,0.0,0.00,0.0,0.0,90836.27
3736,SUCURSAL NORTE,BAE05001125,2025.0,2025-09-16 00:00:00,38,0.0,0.000000,0.0,0.00,0.0,0.0,90836.27
3737,SUCURSAL VALLADOLID,BAE05001125,2025.0,2025-09-16 00:00:00,38,0.0,0.000000,0.0,0.00,0.0,0.0,90836.27
